## Bertweet model for sentiment

### 1. install packages

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 3.0 MB/s 
     |████████████████████████████████| 895 kB 35.4 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 6.6 MB 36.8 MB/s 
     |████████████████████████████████| 596 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import tensorflow
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import pandas as pd
import numpy as np
import io
import os
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
import matplotlib
import matplotlib.pyplot as plt

In [3]:
from transformers.optimization import AdamW
from transformers import get_linear_schedule_with_warmup
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

# For transformers v4.x+:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 2. data preparation

In [6]:
import pandas as pd
# define a function for data preparation
def data_prepare(file_path, lab2ind, tokenizer, max_len = 64, mode = 'train'):
    '''
    file_path: the path to input file. 
                In train mode, the input must be a tsv file that includes two columns where the first is text, and second column is label.
                The first row must be header of columns.

                In predict mode, the input must be a tsv file that includes only one column where the first is text.
                The first row must be header of column.

    lab2ind: dictionary of label classes
    tokenizer: BERT tokenizer
    max_len: maximal length of input sequence
    mode: train or predict
    '''
    # if we are in train mode, we will load two columns (i.e., text and label).
    if mode == 'train':
        # Use pandas to load dataset
        df = pd.read_csv(file_path, delimiter='\t',header=0, names=['content','label'])
        print("Data size ", df.shape)
        labels = df.label.values
        
        # Create sentence and label lists
        labels = [lab2ind[i] for i in labels] 
        print("Label is ", labels[0])
        
        # Convert data into torch tensors
        labels = torch.tensor(labels)

    # if we are in predict mode, we will load one column (i.e., text).
    elif mode == 'predict':
        df = pd.read_csv(file_path, delimiter='\t',header=0, names=['content'])
        print("Data size ", df.shape)
        # create placeholder
        labels = []
    else:
        print("the type of mode should be either 'train' or 'predict'. ")
        return
        
    # Create sentence and label lists
    content = df.content.values
    #### REF START ####

    # We need to add a special token at the beginning for BERT to work properly.
    content = ["[CLS] " + text for text in content]

    # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
    tokenized_texts = [tokenizer.tokenize(text) for text in content]
    
    # if the sequence is longer the maximal length, we truncate it to the pre-defined maximal length
    tokenized_texts = [ text[:max_len+1] for text in tokenized_texts]

    # We also need to add a special token at the end.
    tokenized_texts = [ text+['[SEP]'] for text in tokenized_texts]
    print ("Tokenize the first sentence:\n",tokenized_texts[0])
    
    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    print ("Index numbers of the first sentence:\n",input_ids[0])

    # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
    pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
    input_ids = pad_sequences(input_ids, maxlen=max_len+2, dtype="long", truncating="post", padding="post", value=pad_ind)
    print ("Index numbers of the first sentence after padding:\n",input_ids[0])

    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for pad tokens
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # Convert all of our data into torch tensors, the required datatype for our model
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)
    #### REF END ####

    return inputs, labels, masks    

In [7]:
lab2ind_sentiment = {0: 0, 1: 1, 2:2}

In [8]:
train_inputs_sentiment, train_labels_sentiment, train_masks_sentiment = data_prepare("train_sentiment.tsv", lab2ind_sentiment,tokenizer)

Data size  (45615, 2)
Label is  2
Tokenize the first sentence:
 ['[@@', 'CL@@', 'S@@', ']', '"@@', 'QT', '@@@', 'user', 'In', 'the', 'original', 'draft', 'of', 'the', '7th', 'book@@', ',', 'Rem@@', 'us', 'Lup@@', 'in', 'survived', 'the', 'Battle', 'of', 'Hog@@', 'war@@', 'ts@@', '.', '#HappyBirthday@@', 'Rem@@', 'us@@', 'Lup@@', 'in@@', '"', '[SEP]']
Index numbers of the first sentence:
 [61658, 6411, 381, 317, 61933, 19989, 5238, 4699, 173, 6, 1782, 4271, 15, 6, 4133, 8470, 7, 8867, 148, 30411, 16, 9610, 6, 2606, 15, 60201, 3628, 4813, 4, 11835, 8867, 1924, 30411, 520, 26, 3]
Index numbers of the first sentence after padding:
 [61658  6411   381   317 61933 19989  5238  4699   173     6  1782  4271
    15     6  4133  8470     7  8867   148 30411    16  9610     6  2606
    15 60201  3628  4813     4 11835  8867  1924 30411   520    26     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3

In [9]:
validation_inputs_sentiment, validation_labels_sentiment, validation_masks_sentiment = data_prepare("val_sentiment.tsv", lab2ind_sentiment,tokenizer)

Data size  (2000, 2)
Label is  1
Tokenize the first sentence:
 ['[@@', 'CL@@', 'S@@', ']', 'Dark', 'Souls', '3', 'April', 'Launch', 'Date', 'Confirmed', 'With', 'New', 'Trail@@', 'er:', 'Embrace', 'the', 'dark@@', 'ness@@', '.', '[SEP]']
Index numbers of the first sentence:
 [61658, 6411, 381, 317, 3713, 20987, 163, 1249, 9767, 2553, 22650, 458, 210, 53681, 25570, 26755, 6, 13863, 16571, 4, 3]
Index numbers of the first sentence after padding:
 [61658  6411   381   317  3713 20987   163  1249  9767  2553 22650   458
   210 53681 25570 26755     6 13863 16571     4     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3]


In [10]:
batch_size = 32
# We'll take training samples in random order in each epoch. 
train_data_sentiment = TensorDataset(train_inputs_sentiment, train_masks_sentiment, train_labels_sentiment)
train_dataloader_sentiment = DataLoader(train_data_sentiment, 
                              sampler = RandomSampler(train_data_sentiment), # Select batches randomly
                              batch_size=batch_size)

In [11]:
validation_data_sentiment = TensorDataset(validation_inputs_sentiment, validation_masks_sentiment, validation_labels_sentiment)
validation_dataloader_sentiment = DataLoader(validation_data_sentiment, 
                                   sampler = SequentialSampler(validation_data_sentiment), # Pull out batches sequentially.
                                   batch_size=batch_size)

### 3. Bertweet model

In [12]:
class Bertweet_cls(nn.Module):

    def __init__(self, lab2ind, model_path, hidden_size):
        super(Bertweet_cls, self).__init__()
        self.model_path = model_path
        self.hidden_size = hidden_size
        self.bert_model = AutoModel.from_pretrained(model_path, output_hidden_states=True, output_attentions=True)
        
        self.label_num = len(lab2ind)
        
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.hidden_size, self.label_num)

    def forward(self, bert_ids, bert_mask):
        outputs = self.bert_model(input_ids=bert_ids, attention_mask = bert_mask)
        pooler_output = outputs['pooler_output']
        attentions = outputs['attentions']
        
        x = self.dense(pooler_output)
        x = torch.tanh(x)
        x = self.dropout(x)
        fc_output = self.fc(x)

        return fc_output, attentions

In [13]:
model = Bertweet_cls(lab2ind_sentiment, "vinai/bertweet-base", 768).to(device)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# Parameters for sentiment:
lr = 0.00002
max_grad_norm = 1.0
epochs = 3
warmup_proportion = 0.1
num_training_steps  = len(train_dataloader_sentiment) * epochs
num_warmup_steps = num_training_steps * warmup_proportion

### In Transformers, optimizer and schedules are instantiated like this:
# Note: AdamW is a class from the huggingface library
# the 'W' stands for 'Weight Decay"
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)
# schedules
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

# We use nn.CrossEntropyLoss() as our loss function. 
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


### 4. model traning

In [15]:
def train(model, iterator, optimizer, scheduler, criterion):
    
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        input_ids, input_mask, labels = batch

        outputs,_ = model(input_ids, input_mask)

        loss = criterion(outputs, labels)
        # delete used variables to free GPU memory
        del batch, input_ids, input_mask, labels
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.cpu().item()
        optimizer.zero_grad()
    
    # free GPU memory
    if device == 'cuda':
        torch.cuda.empty_cache()

    return epoch_loss / len(iterator)

In [16]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    all_pred=[]
    all_label = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            input_ids, input_mask, labels = batch

            outputs,_ = model(input_ids, input_mask)
            
            loss = criterion(outputs, labels)

            # delete used variables to free GPU memory
            del batch, input_ids, input_mask
            epoch_loss += loss.cpu().item()

            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(labels.cpu())
    
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return epoch_loss / len(iterator), accuracy, f1score

In [17]:
import os
save_path = './drive/My Drive/Colab Notebooks/ckpt_BERTweet/'
if os.path.exists(save_path) == False:
    os.makedirs(save_path)

In [18]:
# Train the sentiment model
loss_list = []
acc_list = []

for epoch in trange(epochs, desc="Epoch"):
    train_loss = train(model, train_dataloader_sentiment, optimizer, scheduler, criterion)  
    val_loss, val_acc, val_f1 = evaluate(model, validation_dataloader_sentiment, criterion)

    # Create checkpoint at end of each epoch
    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
        }

    torch.save(state, "./drive/My Drive/Colab Notebooks/ckpt_BERTweet/BERT_sentiment_noemoji"+str(epoch+1)+".pt")

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, epochs, train_loss, val_loss, val_acc, val_f1))

Epoch:  33%|███▎      | 1/3 [18:29<36:59, 1109.87s/it]


 Epoch [1/3], Train Loss: 0.6704, Validation Loss: 0.5714, Validation Accuracy: 0.7580, Validation F1: 0.7439


Epoch:  67%|██████▋   | 2/3 [36:58<18:29, 1109.36s/it]


 Epoch [2/3], Train Loss: 0.4770, Validation Loss: 0.5935, Validation Accuracy: 0.7485, Validation F1: 0.7389


Epoch: 100%|██████████| 3/3 [55:27<00:00, 1109.25s/it]


 Epoch [3/3], Train Loss: 0.3672, Validation Loss: 0.6463, Validation Accuracy: 0.7525, Validation F1: 0.7424


### 5. model evalutaion

In [19]:
test_inputs_sentiment, test_labels_sentiment, test_masks_sentiment = data_prepare("test_sentiment.tsv", lab2ind_sentiment,tokenizer)

Data size  (12284, 2)
Label is  1
Tokenize the first sentence:
 ['[@@', 'CL@@', 'S@@', ']', '@@@', 'user', '@@@', 'user', 'what', 'do', 'these', "'@@", '1/2', 'naked', 'pic@@', 's@@', "'", 'have', 'to', 'do', 'with', 'anything@@', '?', 'They@@', "'re", 'not', 'even', 'like', 'that@@', '.', '[SEP]']
Index numbers of the first sentence:
 [61658, 6411, 381, 317, 5238, 4699, 5238, 4699, 66, 32, 198, 1909, 3300, 1842, 7781, 423, 69, 36, 9, 32, 30, 48735, 21, 32752, 81, 46, 132, 43, 6139, 4, 3]
Index numbers of the first sentence after padding:
 [61658  6411   381   317  5238  4699  5238  4699    66    32   198  1909
  3300  1842  7781   423    69    36     9    32    30 48735    21 32752
    81    46   132    43  6139     4     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3]


In [35]:
batch_size = 32
# We'll take training samples in random order in each epoch. 
test_data_sentiment = TensorDataset(test_inputs_sentiment, test_masks_sentiment, test_labels_sentiment)
test_dataloader_sentiment = DataLoader(test_data_sentiment, 
                              sampler = RandomSampler(test_data_sentiment), # Select batches randomly
                              batch_size=batch_size)

In [36]:
best_sentiment_checkpoint = torch.load("./drive/My Drive/Colab Notebooks/ckpt_BERTweet/BERT_sentiment_noemoji1.pt",map_location=torch.device('cpu'))

In [37]:
model_sentiment = Bertweet_cls(lab2ind_sentiment, "vinai/bertweet-base", 768).to(device)
model_sentiment.load_state_dict(best_sentiment_checkpoint['state_dict'])

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [38]:
test_loss, test_acc, test_f1 = evaluate(model_sentiment, test_dataloader_sentiment, criterion)
print('Test Loss: {:.4f}, Test Accuracy: {:.4f}, Test F1: {:.4f}'.format(test_loss, test_acc, test_f1))

Test Loss: 0.6050, Test Accuracy: 0.7199, Test F1: 0.7205


Test on vaccine and masks

In [39]:
lab2ind_vac = {'negative': 0, 'neutral': 1, 'positive': 2} 

In [40]:
vac_inputs_sentiment, vac_labels_sentiment, vac_masks_sentiment = data_prepare("test_vaccines.tsv", lab2ind_vac,tokenizer)

Data size  (101, 2)
Label is  1
Tokenize the first sentence:
 ['[@@', 'CL@@', 'S@@', ']', 'According', 'to', 'the', 'latest', '#v@@', 'acc@@', 'ine', 'report', 'by', 'the', '#UK@@', 'Health@@', 'Sec@@', 'urity@@', 'Ag@@', 'ency@@', ',', '#V@@', 'acc@@', 'ine@@', 'Eff@@', 'ectiveness', 'for', 'tri@@', 'ple-@@', 'jab@@', 'bed', 'young', 'and', 'older', 'people', 'are', 'below', 'zero@@', ',', 'and', 'more', 'likely', 'to', 'get', 'with', '#Co@@', 'vid', '[SEP]']
Index numbers of the first sentence:
 [61658, 6411, 381, 317, 3877, 9, 6, 1405, 6615, 6530, 1466, 1649, 61, 6, 20461, 15251, 11060, 43243, 7709, 35615, 7, 2863, 6530, 3800, 24037, 61598, 19, 3203, 35939, 27819, 424, 859, 13, 1879, 83, 41, 2703, 46631, 7, 13, 89, 2056, 9, 51, 30, 8232, 4759, 3]
Index numbers of the first sentence after padding:
 [61658  6411   381   317  3877     9     6  1405  6615  6530  1466  1649
    61     6 20461 15251 11060 43243  7709 35615     7  2863  6530  3800
 24037 61598    19  3203 35939 27819   424

In [41]:
batch_size = 32
# We'll take training samples in random order in each epoch. 
vac_data_sentiment = TensorDataset(vac_inputs_sentiment, vac_masks_sentiment, vac_labels_sentiment)
vac_dataloader_sentiment = DataLoader(vac_data_sentiment, 
                              sampler = RandomSampler(vac_data_sentiment), # Select batches randomly
                              batch_size=batch_size)

In [43]:
best_sentiment_checkpoint = torch.load("./drive/My Drive/Colab Notebooks/ckpt_BERTweet/BERT_sentiment_noemoji1.pt",map_location=torch.device('cpu'))

In [44]:
model_sentiment = Bertweet_cls(lab2ind_vac, "vinai/bertweet-base", 768).to(device)
model_sentiment.load_state_dict(best_sentiment_checkpoint['state_dict'])

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [45]:
criterion = nn.CrossEntropyLoss()
test_loss, test_acc, test_f1 = evaluate(model_sentiment, vac_dataloader_sentiment, criterion)
print('Test Loss: {:.4f}, Test Accuracy: {:.4f}, Test F1: {:.4f}'.format(test_loss, test_acc, test_f1))

Test Loss: 0.4796, Test Accuracy: 0.7822, Test F1: 0.7920


Test on masks

In [46]:
mask_inputs_sentiment, mask_labels_sentiment, mask_masks_sentiment = data_prepare("test_masks.tsv", lab2ind_vac,tokenizer)

Data size  (90, 2)
Label is  2
Tokenize the first sentence:
 ['[@@', 'CL@@', 'S@@', ']', 'Thank', 'you', 'to', 'all', 'those', 'who', 'still', 'limit', 'indoor', 'social', 'gather@@', 'ings@@', ',', 'get', 'boo@@', 'sted@@', ',', 'mask', 'at', 'the', 'grocery', 'store', '&@@', 'amp@@', ';', 'send', 'their', 'children', 'to', 'school', 'in', 'mask@@', 's.', 'I', 'see', 'you', 'and', 'appreciate', 'you', 'caring', 'about', 'the', 'safety', 'of', 'the', 'rest', 'of', 'us@@', '.', 'We', 'over', 'me@@', '.', '#COVID19', '#Covid@@', 'Is@@', 'Not@@', 'Over', '#M@@', 'ask@@', 'Up', '[SEP]']
Index numbers of the first sentence:
 [61658, 6411, 381, 317, 396, 14, 9, 48, 268, 87, 135, 3947, 17617, 1009, 62780, 17726, 7, 51, 4910, 41756, 7, 7146, 35, 6, 8923, 1297, 19295, 6755, 208, 786, 130, 994, 9, 230, 16, 61163, 32188, 8, 95, 14, 13, 1621, 14, 5325, 62, 6, 3705, 15, 6, 765, 15, 1924, 4, 134, 141, 1677, 4, 4270, 60605, 3166, 8719, 1773, 1230, 11686, 857, 3]
Index numbers of the first sentence af

In [47]:
batch_size = 32
# We'll take training samples in random order in each epoch. 
mask_data_sentiment = TensorDataset(mask_inputs_sentiment, mask_masks_sentiment, mask_labels_sentiment)
mask_dataloader_sentiment = DataLoader(mask_data_sentiment, 
                              sampler = RandomSampler(mask_data_sentiment), # Select batches randomly
                              batch_size=batch_size)

In [48]:
test_loss, test_acc, test_f1 = evaluate(model_sentiment, mask_dataloader_sentiment, criterion)
print('Test Loss: {:.4f}, Test Accuracy: {:.4f}, Test F1: {:.4f}'.format(test_loss, test_acc, test_f1))

Test Loss: 0.6824, Test Accuracy: 0.6556, Test F1: 0.6197


### 6. prediction

In [ ]:
def sententce_prepocess(content, tokenizer, max_len = 64):
    """
    content: list of string. Each string is a sample. We only include one sample in this list.
    tokenizer: BertTokenizerFast
    """
    #### REF START ####

    # We need to add a special token at the beginning for BERT to work properly.
    content = ["[CLS] " + text for text in content]

    # Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
    tokenized_texts = [tokenizer.tokenize(text) for text in content]
    
    # if the sequence is longer the maximal length, we truncate it to the pre-defined maximal length
    tokenized_texts = [ text[:max_len+1] for text in tokenized_texts]

    # We also need to add a special token at the end.
    tokenized_texts = [ text+['[SEP]'] for text in tokenized_texts]
    #print ("Tokenize the first sentence:\n",tokenized_texts[0])
    
    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    #print ("Index numbers of the first sentence:\n",input_ids[0])

    # Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
    pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
    input_ids = pad_sequences(input_ids, maxlen=max_len+2, dtype="long", truncating="post", padding="post", value=pad_ind)
    #print ("Index numbers of the first sentence after padding:\n",input_ids[0])

    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for pad tokens
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # Convert all of our data into torch tensors, the required datatype for our model
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)
    #### REF END ####

    return tokenized_texts, inputs, masks

In [ ]:
masks_df = pd.read_csv('masks_twitter.csv')
masks_df.head()

In [ ]:
emoji_result = []
emoji_confid = []
for index, row in masks_df.iterrows():
    tokenized_texts, input_ids, masks = sententce_prepocess([row['tweet_text']], tokenizer)
    input_ids, masks = input_ids.to(device), masks.to(device)
    outputs,_ = model_sentiment(input_ids, masks)
    probabilities, predicted = torch.max(outputs.cpu().data, 1)
    emoji_result.append(predicted.item())
    emoji_confid.append(probabilities.item())
